# LABORATORIO #1 - Deteccion de Phishing
 Autor: Javier Estuardo 19202
 
 Autor: Marco Ramirez   19588


### Parte 1


* 1. Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones. 

In [7]:
import pandas as pd

df = pd.read_csv('dataset_pishing.csv')
df.sample(n=5).head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
6164,https://en.wikipedia.org/wiki/Lists_of_network...,0,0,0,0,0,0,0,0,0,...,0,0,0,902,7133,12,0,0,7,legitimate
1250,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,1,0,0,15,4003,5816617,0,1,0,phishing
7889,https://michaelannedye.files.wordpress.com/201...,0,0,0,0,0,0,0,0,0,...,1,1,0,587,7448,385,0,0,8,legitimate
1127,http://www.riblanda.com/support.ionos.com/clie...,0,1,1,0,0,0,0,1,0,...,1,0,0,23,707,0,0,1,1,phishing
4573,http://www.pollsbee.com/wp-content/Doc/9600588...,1,1,0,0,0,0,0,0,0,...,1,0,0,322,43,0,0,1,0,phishing


* 2. Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como 
“pishing”. ¿Está balanceado el dataset? 

In [8]:
print(df['status'].value_counts())

legitimate    5715
phishing      5715
Name: status, dtype: int64


                El dataset se encuentra balanceado.

* 3. Para ello escriba las funciones necesarias y genere las nuevas columnas del dataset. Muestre un nuevo 
ejemplo de cinco observaciones donde se visualicen algunas de las columnas nuevas. 

In [23]:
from urllib.parse import urlparse

def url_length(url):
    return len(url)

def hostname_length(url):
    parsed_uri = urlparse(url)
    hostname = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    return len(hostname)

def special_characters(url):
    special_char = 0
    for i in range(0, len(url)):
        ch = url[i]
        if (url[i].isalpha()):
            continue
        elif (url[i].isdigit()):
            continue
        else: 
            special_char += 1
    return special_char

def https_token(url):
    if 'https' in url:
        return 1
    else:
        return 0

def ratio_of_digits_in_url(url):
    digits = sum(c.isdigit() for c in url)
    urlen = url_length(url)
    try:
        ratio = digits/urlen
    except:
        ratio = 0  
    return ratio

def ratio_of_digits_in_hostname(url):
    parsed_uri = urlparse(url)
    hostname = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    hostnamelen = hostname_length(url)
    digits = sum(c.isdigit() for c in hostname)
    try:
        ratio = digits/hostnamelen
    except:
        ratio = 0  
    return ratio

new_df = df.assign(url_len = df['url'].apply(url_length))
new_df2 = new_df.assign(hostname_len = df['url'].apply(hostname_length))
new_df3 = new_df2.assign(special_chars = df['url'].apply(special_characters))
new_df4 = new_df3.assign(https = df['url'].apply(https_token))
new_df5 = new_df4.assign(digits_url = df['url'].apply(ratio_of_digits_in_url))
new_df6 = new_df5.assign(digits_hostname = df['url'].apply(ratio_of_digits_in_hostname))
new_df6.sample(n=5).head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,dns_record,google_index,page_rank,status,url_len,hostname_len,special_chars,https,digits_url,digits_hostname
3291,http://hutoknepper.de/,0,0,0,0,0,0,0,0,0,...,0,0,0,phishing,22,22,5,0,0.000000,0.000000
646,http://en.academic.ru/dic.nsf/enwiki/1962402,1,0,0,0,0,0,0,0,0,...,0,0,3,legitimate,44,22,9,0,0.159091,0.000000
2445,http://www.kievskaya22.ru/44302557590lochttp3A...,1,1,0,0,1,0,0,0,0,...,0,1,0,phishing,165,26,12,0,0.236364,0.076923
7779,http://www.medicinenet.com/peripheral_vascular...,0,1,0,0,0,0,0,0,0,...,0,0,6,legitimate,66,27,10,0,0.000000,0.000000
5298,http://www.pure-mac.com/ftp.html,0,1,0,0,0,0,0,0,0,...,0,0,5,legitimate,32,24,8,0,0.000000,0.000000


* 4. Realice  las  modificaciones  necesarias  para  convertir  la  variable  categórica  status  a  una  variable 
binaria. Elimine la columna del dominio. 

* 5. Genere  un  reporte  de  perfil  con  la  librería  pandas_profiling.  Analice  el  reporte  y  determine  las 
columnas que son constantes, o que no tienen una varianza alta con la columna status. Almacene su 
reporte como una página html. 

* 6. En base al análisis del reporte, elimine las características repetidas o irrelevantes para la clasificación 
de un sitio de pishing. Verifique que no posee observaciones repetidas. 

### Parte 2

* 1. Separación de datos

        * Datos de entrenamiento: 55%
 
        * Datos de validación: 15% 
        * Datos de prueba: 30% 
        * Almacene cada dataset como un archivo .csv 


* 2. Utilice dos algoritmos de Machine Learning para entrenar el modelo. Muestre y explique los valores 
obtenidos de las siguientes métricas para los datos de validación y pruebas, para cada modelo, en 
base al contexto del problema (detección de Pishing). 

        * Modelo 1

        * Resultados de modelo 1

        * Modelo 2


        * Resultados del modelo 2

### Discusion

* 1. ¿Cuál es el impacto de clasificar un sitio legítimo como Pishing?  
* 2. ¿Cuál es el impacto de clasificar un sitio de Pishing como legítimo? 
* 3. En base a las respuestas anteriores, ¿Qué métrica elegiría para comparar modelos similares de clasificación de pishing? 
* 4. ¿Qué modelo es mejor para la clasificación de Pishing? Justifique 
* 5. En base a las métricas obtenidas ¿es necesaria la intervención de una persona humana para tomar la decisión final? 